## Feature pre-procession and hyper-parameter tuning lab

In this lab we continue to use Ames housing dataset to practice the model optimization techniques.
There are 3 places need to be completed:
1. Scale features
2. Regularize features
3. Find the best hyper-parameter alpha

In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import sys

%matplotlib inline

## Load the data

We will use the Ames housing dataset from http://ww2.amstat.org/publications/jse/v19n3/decock.pdf.

In [ ]:
data = pd.read_csv('AmesHousing.txt', delimiter='\t')
numeric_values = np.where(
    (data.dtypes == np.dtype('int64'))
    | (data.dtypes == np.dtype('float64'))
)[0]
X = data[numeric_values[2:-1]].values
y = data['SalePrice'].values
feature_names = data.columns[numeric_values[2:-1]]

In [ ]:
numeric_values

## Split the data into train / test

In [ ]:
from sklearn.model_selection import train_test_split

X = np.nan_to_num(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

## Inspect the data

Let's take a quick look at the distribution of variables, looking for potential sources of outliers or features that may reduce the performance of our model.

We'll use whisker plots to visualize all of the features at once. The organge line is the median value, the edges of the box denote the 1st and 3rd quartile, the bars at the end denote the maxium and minimum values, and individual points within the top and bottom quartile.

In [ ]:
fig, ax = plt.subplots(figsize=(15,5))
ax.boxplot(X_train)
ax.set_xticks(range(1, X_train.shape[1] + 1))
_ = ax.set_xticklabels(feature_names, rotation=60)

## Scale features
The scales of different features have a large variance. It could affect model performance. Rescale the range of each feature to [0, 1]. 

For each feature, use formula 
```
x_scaled = (x - min) / (max - min)
```
where min and max are the minimum and maximum values of the feature in the training set.

In [ ]:
scaler = preprocessing.MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Inspect the data after scaling
fig, ax = plt.subplots(figsize=(15,5))
ax.boxplot(X_train_scaled)
ax.set_xticks(range(1, X_train_scaled.shape[1] + 1))
_ = ax.set_xticklabels(feature_names, rotation=60)

## Train a model



In [ ]:
# Without scaling

clf = LinearRegression()
clf.fit(X_train, y_train)
fit_r2 = clf.score(X_train, y_train)
y_pred = clf.predict(X_test)
mse_pred = mean_squared_error(y_test, y_pred)

fig, ax = plt.subplots()
y_min = np.min(y_test)
y_max = np.max(y_test)
ax.plot([y_min, y_max], [y_min, y_max], 'k--')
ax.plot(y_test, y_pred, 'o', alpha=0.3)
ax.set_xlabel('True Value')
ax.set_ylabel('Predicted Value')

print('Model Coefficients')
for fn, coef in zip(feature_names, clf.coef_):
    print('%10s: %.2f' % (fn, coef))
print('\nFit R^2 = %.2f, prediction MSE = %.5f' % (fit_r2, mse_pred))

In [ ]:
# With scaling

clf = LinearRegression()
clf.fit(X_train_scaled, y_train)
fit_r2 = clf.score(X_train_scaled, y_train)
y_pred = clf.predict(X_test_scaled)
scaled_mse_pred = mean_squared_error(y_test, y_pred)

fig, ax = plt.subplots()
y_min = np.min(y_test)
y_max = np.max(y_test)
ax.plot([y_min, y_max], [y_min, y_max], 'k--')
ax.plot(y_test, y_pred, 'o', alpha=0.3)
ax.set_xlabel('True Value')
ax.set_ylabel('Predicted Value')

print('Model Coefficients')
for fn, coef in zip(feature_names, clf.coef_):
    print('%10s: %.2f' % (fn, coef))
print('\nFit R^2 = {0:.2f}, prediction MSE = {1:.5f}, prediction MSE reduction = {2:.5f}%'.format(
      fit_r2, scaled_mse_pred, (mse_pred - scaled_mse_pred) / mse_pred * 100))

## Regularize features

Use lasso with L1 norm to penalize non-zero coefficients. It can effectively reduce the number of variables on which the model is dependent.

Objective function of lasso:
```
(1 / (2 * n_samples)) * ||y - Xw||^2_2 + alpha * ||w||_1
```

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=90)
lasso.fit(X_train_scaled, y_train)
lasso_fit_r2 = lasso.score(X_train_scaled, y_train)
y_pred = lasso.predict(X_test_scaled)
lasso_mse_pred = mean_squared_error(y_test, y_pred)

fig, ax = plt.subplots()
y_min = np.min(y_test)
y_max = np.max(y_test)
ax.plot([y_min, y_max], [y_min, y_max], 'k--')
ax.plot(y_test, y_pred, 'o', alpha=0.3)
ax.set_xlabel('True Value')
ax.set_ylabel('Predicted Value')

print('Model Coefficients')
for fn, coef in zip(feature_names, lasso.coef_):
    print('%10s: %.2f' % (fn, coef))
print('\nFit R^2 = {0:.2f}, prediction MSE = {1:.5f}, prediction MSE reduction = {2:.5f}%'.format(
      lasso_fit_r2, lasso_mse_pred, (mse_pred - lasso_mse_pred) / mse_pred * 100))

## Find the best alpha

Use grid search to find the best alpha in lasso that gives the minimum test error.

In [ ]:
alphas = [i for i in range(0, 200, 10)]
mse_pred_array = []
best_mse = sys.maxint
best_alpha = 0
for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train_scaled, y_train)
    y_pred = lasso.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    mse_pred_array.append(mse)
    if mse < best_mse:
        best_mse = mse
        best_alpha = alpha

fig, ax = plt.subplots()
y_min = np.min(mse_pred_array)
y_max = np.max(mse_pred_array)
ax.plot(alphas, mse_pred_array, '*-')
ax.set_xlabel('Alpha')
ax.set_ylabel('MSE')

print('\nBest alpha = {0}, best prediction MSE = {1:.5f}, reduction = {2:.4f}%'.format(
    best_alpha, best_mse, (mse_pred - best_mse) / mse_pred * 100))

## Bonus Point

* Try other normalization methods and compare the difference. 
* Convert the non-numeric features to binary features.